In [ ]:
# Starts the autoreload extension, which allows editing the .py files with the notebook running and automatically imports the latest changes

%load_ext autoreload
%autoreload 2

import trim_duplicates, model, network, utils, gradcam, plots
from dataset import Dataset
import haiku as hk
import jax.numpy as jnp
import numpy as np
import jax

In [ ]:
assert jax.local_device_count() >= 8

In [ ]:
NUM_CLASSES = 4
SEED = 12
BATCH_SIZE = 128

In [ ]:
rng = jax.random.PRNGKey(SEED)
dataset_tawsifur = Dataset.load("tawsifur_kaggle", rng=rng)

In [ ]:
net, optim = model.init_net_and_optim(dataset_tawsifur.x_train, NUM_CLASSES, BATCH_SIZE)

# Gets functions for the model
net_container = network.create(net, optim, BATCH_SIZE, shape = (10, 256, 256, 3))

In [ ]:
def basemodel_process(x): return x
basemodel_tawsifur = model.train_model("basemodel_tawsifur", net_container, basemodel_process, dataset_tawsifur)
y_test_pred_tawsifur = net_container.predict(basemodel_tawsifur.params, basemodel_tawsifur.state, dataset_tawsifur.x_test)
plots.confusion_matrix(dataset_tawsifur, y_test_pred_tawsifur, "Tawsifur - Not curated")

In [ ]:
sims = trim_duplicates.compute_similarities(dataset_tawsifur, net_container, basemodel_tawsifur)

In [ ]:
trim_duplicates.plot_similarities(dataset_tawsifur, sims, threshold=0.99)
dataset_tawsifur_curated = trim_duplicates.remove_duplicates(dataset_tawsifur, sims, threshold=0.99)

In [ ]:
print(dataset_tawsifur_curated.x_train.shape, dataset_tawsifur.x_train.shape)

In [ ]:
basemodel_tawsifur_curated = model.train_model("basemodel_tawsifur_curated", net_container, basemodel_process, dataset_tawsifur_curated)
y_pred_tawsifur_curated = net_container.predict(basemodel_tawsifur_curated.params, basemodel_tawsifur_curated.state, dataset_tawsifur_curated.x_test)
plots.confusion_matrix(dataset_tawsifur_curated, y_pred_tawsifur_curated, "Tawsifur - Curated")